In [21]:
import json
from collections import Counter
import pickle
import numpy as np
import pandas as pd
import os
import sys
import argparse
from os.path import dirname, realpath

import math
import hashlib
import datetime
import datetime as datetime
sys.path.append("./src/")

import torch
from torch import nn, optim
from torch.nn import functional as F

from sklearn.calibration import calibration_curve
from sklearn.linear_model import LogisticRegression
from sklearn.base import BaseEstimator
sys.path.append(dirname(dirname(realpath(os.getcwd()))))

from scipy.optimize import minimize_scalar
from scipy.optimize import minimize
from sklearn.metrics import log_loss
import matplotlib.pyplot as plt
src_path = "G:\\FillmoreCancerData\\markhe\\VTERisk" 



In [30]:
TRAIN_OUT = 'CSV_for_R/totbM_tr_1_28.csv'
DEV_OUT = 'CSV_for_R/totbM_dev_1_28.csv'
TEST_OUT = 'CSV_for_R/Mb_test_1_28.csv'

TEST_OUT1 = 'CSV_for_R/Mb1_test_1_28.csv'
TEST_OUT2 = 'CSV_for_R/Mb2_test_1_28.csv'
TEST_OUT3 = 'CSV_for_R/Mb3_test_1_28.csv'
TEST_OUT4 = 'CSV_for_R/Mb4_test_1_28.csv'

In [23]:
trainpath = os.path.join(src_path ,'logs_transformer_vte/L/1_28/', "1ef57026b57379a05f64994f9760525e.results.train_preds")
devpath = os.path.join(src_path ,'logs_transformer_vte/L/1_28/', "1ef57026b57379a05f64994f9760525e.results.dev_preds")
testpath = os.path.join(src_path ,'logs_transformer_vte/L/1_28/', "1ef57026b57379a05f64994f9760525e.results.test_preds")


In [24]:
def md5(key):
    return hashlib.md5(repr(key).encode()).hexdigest()
testDF = pd.read_csv(os.path.join(src_path, 'Notebooks/Find/fixed_dx.csv'))
pat_ids = (testDF['patient_id'] ).astype(int)
testDF['pids'] = pat_ids.apply(md5)

In [25]:
def get_4Ms(M):
    M1 = M.drop_duplicates(['patient_id'])
    m0 = M.duplicated(['patient_id'])
    M234 = M[m0]
    M2 = M234.drop_duplicates(['patient_id'])

    m00 = M234.duplicated(['patient_id'])
    M34 = M234[m00]
    M3 = M34.drop_duplicates(['patient_id'])

    m000 = M34.duplicated(['patient_id'])
    M4 = M34[m000]
    return M1, M2, M3, M4

In [26]:
tdf = pd.DataFrame({'patient_id': (testDF['patient_id'] ),    
                    'pids': (testDF['pids'] ),    
                    'dob': (testDF['dob'] ),  
                    'outcome_date': (testDF['outcome_date'] ), 
                    'obs_time_end': (testDF['obs_time_end'] ),  
                    'index_date': (testDF['index_date'] ),  
                    'diag_date': (testDF['diag_date'] ),  
                    'outcome': (testDF['outcome'])    })

In [27]:
with open(trainpath, 'rb') as f:
    R = pickle.load(f)
    p = np.array(R['probs'])

Df = pd.DataFrame.from_dict(R)
Df['probs'] = Df['probs'].astype(float)
Df['exams'] = Df['exams'].astype(int)
M = pd.merge(tdf, Df)
M.to_csv(TRAIN_OUT) 

In [28]:
with open(devpath, 'rb') as f:
    R_d = pickle.load(f)

Df_dev = pd.DataFrame.from_dict(R_d)
Df_dev['probs'] = Df_dev['probs'].astype(float)
M_dev = pd.merge(tdf, Df_dev)
M_dev.to_csv(DEV_OUT) 

In [31]:
with open(testpath, 'rb') as f:
    R_t = pickle.load(f)

Df_test = pd.DataFrame.from_dict(R_t)
Df_test['probs'] = Df_test['probs'].astype(float)
M_test = pd.merge(tdf, Df_test)
M_test.to_csv(TEST_OUT)

#M_test1, M_test2, M_test3, M_test4 = get_4Ms(M_test)
#M_test1.to_csv(TEST_OUT1) 
#M_test2.to_csv(TEST_OUT2) 
#M_test3.to_csv(TEST_OUT3) 
#M_test4.to_csv(TEST_OUT4)

In [32]:
TEST_OUT

'CSV_for_R/Mb_test_1_28.csv'